In [2]:
import pandas as pd
import pulp

DemandXY = pd.read_csv("town_data.csv")
SupplyXY = pd.read_csv("hospital_data.csv")

model = pulp.LpProblem(name="Maximal_Cover", sense=pulp.LpMaximize)

SUPPLY_XY_COUNT = len(SupplyXY)
DEMAND_XY_COUNT = len(DemandXY)
MARGINAL_DISTANCE = 30000
CAPACITY = 3000
HosCnt = 5

#Define Variables
#var_x는 demand위치 와 supply병원갯수의 관계를 정의
#ex) demand위치가 3개 supply병원이 5개면
#[[1->1, 1->2, 1->3, 1->4, 1->5],
#[2->1, 2->2, 2->3, 2->4, 2->5],
#[3->1, 3->2, 3->3, 3->4, 3->5]]
var_x = [[pulp.LpVariable(name=f"D{i}~S{j}",lowBound=0) for j in range(1,SUPPLY_XY_COUNT+1)] for i in range(1,DEMAND_XY_COUNT + 1)]
#? var_y는 Supply 병원 설치여부(Binary) 변수
var_y = [pulp.LpVariable(name=f"S{i}",lowBound=0, upBound=1,cat="Binary") for i in range(1,SUPPLY_XY_COUNT+1)]

#Object Function
#최대화하는 목적식 설정
exp = pulp.LpAffineExpression()
for i in range(DEMAND_XY_COUNT):
    for j in range(SUPPLY_XY_COUNT):
        dist = ((DemandXY.iloc[i,0] - SupplyXY.iloc[j,0])**2 + (DemandXY.iloc[i,1] - SupplyXY.iloc[j,1])**2)**0.5
        if dist < MARGINAL_DISTANCE:
            exp += var_x[i][j]
model += exp

#Constraints
#일단 demand위치기준에서 supply병원까지 거리가 MARGINAL_DISTANCE 이내인 변수들을 찾기.
#demand위치->supply병원 거리가 가능한 모든 경우를 식에 더하고  
#demand는 supply병원까지 여러곳 거리가 다 가능해도 demand는 정해져있으니까  <= demand  수식을 설정.
for i in range (DEMAND_XY_COUNT):
    exp = pulp.LpAffineExpression()
    for j in range(SUPPLY_XY_COUNT):
        dist = ((DemandXY.iloc[i,0] - SupplyXY.iloc[j,0])**2 + (DemandXY.iloc[i,1] - SupplyXY.iloc[j,1])**2)**0.5
        if dist < MARGINAL_DISTANCE:
            exp += var_x[i][j]
    model += exp <= DemandXY.iloc[i,2]

#논리조건 설정(Binary)
#supply병원기준으로 for문 시작 -> supply병원기준으로 거리가 가능한 demand위치들을 식에 추가하고,
#supply병원이 설치된다면 수용가능한 CAPACITY를 추가한다.(Binary) 
#설치안하면 (0 * 3000)명 수용가능, 설치하면 (1 * 3000)명 수용가능
for j in range(SUPPLY_XY_COUNT):
    exp = pulp.LpAffineExpression()
    for i in range (DEMAND_XY_COUNT):
        dist = ((DemandXY.iloc[i,0] - SupplyXY.iloc[j,0])**2 + (DemandXY.iloc[i,1] - SupplyXY.iloc[j,1])**2)**0.5
        if dist < MARGINAL_DISTANCE:
            exp += var_x[i][j]
    exp += -1 * CAPACITY * var_y[j]
    model += exp <= 0

#설치할 Supply병원의 갯수를 제한하는 제약식 추가
exp = pulp.LpAffineExpression()
for j in range(SUPPLY_XY_COUNT):
    exp += var_y[j]
model += exp <= HosCnt


model.writeLP('out.txt')

model.solve()

for variable in var_y:
    if variable.value() > 0:
        print(f"{variable.name} 병원: {variable.value()}")

print(f"Optival objective value: {model.objective.value()}")

S1 병원: 1.0
S15 병원: 1.0
S19 병원: 1.0
S20 병원: 1.0
S90 병원: 1.0
Optival objective value: 14999.9999843
